In [ ]:
import pandas as pd
import loader as load
# aak_ge = load.loadGEWithClinical()
aak_ge = None

In [ ]:
import processor as pr

tcma_cancer_type = ["COAD", "ESCA", "HNSC", "READ", "STAD"]

# data = [load.loadTCMA("genus"), load.loadTCMA("phylum"), aak_ge, load.loadGEOverlappingTCMA("genus"), load.loadGEOverlappingTCMA("phylum")]
# source_types = ["GENUS", "PHYLUM", "GE", "GENUS n GE", "PHYLUM n GE"]
# file_names = ["TCMA_Genus", "TCMA_Phylum", "TCGA_GE(Aakash)", "TCMA_Genus_GE(Aakash)_Overlap", "TCMA_Phylum_GE(Aakash)_Overlap"]
source_types = ["GENUS", "GE", "GENUS n GE"]

diagnostic ="stage"
samples_pool = "all"


data, file_names = load.loadAll(includeStage=(diagnostic=="stage"), sameSamples=(samples_pool=="intersect"))


In [ ]:
# data[2].loc["TCGA-A6-2682-01"]["stage"]
# d = data[2]
# d[d["project"]=="READ"]
t = load.attachStageStatus(data[1])
print(file_names[1])
print(t[t.project.isin(tcma_cancer_type)].groupby(["project"])["stage"].value_counts())

In [ ]:
reduction_type = "PCA"

In [ ]:
import processor as pr

for feature_amount in [0, 5, 10]:
    for i, b in enumerate(data):
        X, Y, titles = [], [], []

        if diagnostic == "tumor":
            b = load.attachTumorStatus(b)
        else:
            b = load.attachStageStatus(b)

        for c in tcma_cancer_type:
            x, y = pr.splitData(b, diagnostic, project=c)
            
            best_x = pr.selectFeatures(x, y, feature_amount)
            x = best_x

            if reduction_type == "PCA":
                X.append(pr.getPCA(x))
            else:
                X.append(pr.getTSNE(x))
            Y.append(y)
            titles.append(f"{c} {source_types[i]} {reduction_type} p:{feature_amount}")
        pr.plotScatter(X, Y, titles, f"Visual/{diagnostic}/{samples_pool}/{reduction_type}/p{feature_amount}/{file_names[i]}", diagnostic=diagnostic)